# Goals

Understanding why and when employees are most likely to leave. Data set with info about the employees and have to predict when employees are going to quit by understanding the main drivers of employee churn.

In [1]:
%matplotlib inline

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

import seaborn as sns

PATH = Path('data')

In [4]:
from itertools import product

In [5]:
df = pd.read_csv(PATH/'employee_retention_data.csv',parse_dates=['join_date', 'quit_date'])

Assume, for each company, that the headcount starts from zero on 2011/01/23. Estimate employee headcount, for each company, on each day, from 2011/01/24 to 2015/12/13. That is, if by 2012/03/02 2000 people have joined company 1 and 1000 of them have already quit, then company headcount on 2012/03/02 for company 1 would be 1000.
You should create a table with 3 columns: day, employee_headcount, company_id.
What are the main factors that drive employee churn? Do they make sense? Explain your findings.
If you could add to this data set just one variable that could help explain employee churn, what would that be?

In [6]:
df.head()
df.shape

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaT
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


(24702, 7)

In [7]:
df.join_date.describe()
df.quit_date.describe()

count                   24702
unique                    995
top       2012-01-03 00:00:00
freq                      105
first     2011-01-24 00:00:00
last      2015-12-10 00:00:00
Name: join_date, dtype: object

count                   13510
unique                    664
top       2015-05-08 00:00:00
freq                      111
first     2011-10-13 00:00:00
last      2015-12-09 00:00:00
Name: quit_date, dtype: object

# Headcount table

In [8]:
dates=pd.date_range('2011-01-24','2015-12-23').values
companies=sorted(df.company_id.unique())

In [9]:
df_headcount=pd.DataFrame(list(product(*[dates,companies])),columns=['date','company_id'])
df_headcount.date = pd.to_datetime(df_headcount.date)

In [10]:
df_headcount.head()

,date,company_id
0,2011-01-24,1
1,2011-01-24,2
2,2011-01-24,3
3,2011-01-24,4
4,2011-01-24,5


In [11]:
df_join=df.groupby(['join_date','company_id']).size().reset_index(name='join_count')
df_headcount = df_headcount.merge(df_join,how='left',
                                  left_on=['date','company_id'],
                                  right_on=['join_date','company_id']).drop('join_date',axis=1)

In [12]:
df_headcount.head()

,date,company_id,join_count
0,2011-01-24,1,25.0
1,2011-01-24,2,17.0
2,2011-01-24,3,9.0
3,2011-01-24,4,12.0
4,2011-01-24,5,5.0


In [13]:
df_quit = df.groupby(['quit_date','company_id']).size().reset_index(name='quit_count')

In [14]:
df_quit = df.groupby(['quit_date','company_id']).size().reset_index(name='quit_count')
df_headcount = df_headcount.merge(df_quit,how='left',
                                  left_on=['date','company_id'],
                                  right_on=['quit_date','company_id']).drop('quit_date',axis=1)

In [15]:
df_headcount.head()

,date,company_id,join_count,quit_count
0,2011-01-24,1,25.0,NaN
1,2011-01-24,2,17.0,NaN
2,2011-01-24,3,9.0,NaN
3,2011-01-24,4,12.0,NaN
4,2011-01-24,5,5.0,NaN


In [16]:
df_headcount.fillna(0,inplace=True)

In [18]:
df_headcount['head_count'] =df_headcount.join_count - df_headcount.quit_count
for col in df_headcount.columns.values[-3:]:
    df_headcount[col] = df_headcount[col].astype(np.int32)

In [21]:
df_headcount.head()

,date,company_id,join_count,quit_count,head_count
0,2011-01-24,1,25,0,25
1,2011-01-24,2,17,0,17
2,2011-01-24,3,9,0,9
3,2011-01-24,4,12,0,12
4,2011-01-24,5,5,0,5


In [24]:
df_headcount.date.nunique()

1795